# Хакатон "Сибур". Команда Hack.zamAI

- метрики изношенности (пробег каждого вагона)

- почему нельзя все типы ремонта за одну ближайщую дату?
- время на ремонт?
- 

### Задача "Вагоны"  

Разработать комплексное решение, помогающее специалисту по расчету календаря отправки вагонов на плановое тех.обслуживание оптимизировать логистические затраты.

Решение должно быть полезно для текущей работы сотрудника, упрощать его работу, помогать принимать оперативные решения.

Решение должно состоять из двух частей:
1. Алгоритм на основе анализа данных, предсказывающий оптимальные времена и депо для ремонтов.
2. Интерфейс с визуализацией данных и результатов работы алгоритма и любых дополнительных данных. Это может быть web, mob app, bot и другие формы.

Главный комментарий: это реальный кейс, решение должно быть полезно реальному человеку с реальными должностными обязанностями в реальных рабочих условиях.

Для решения задачи командам будет предложено:
1. Датасет об отправке на ремонт 18000 вагонов за несколько лет с данными о тайминге, расстояниях и т.д.
https://drive.google.com/open?id=1FGFVbgqe5QWilpiyVG9z3AhVRN7Bt_Mp

Уточнения к датасету: 
1.1 перед каждым ремонтом нужно потратить 3 000 000 р  на подготовку вагона. При этом считается, что если делается 2 ремонта подряд, то достаточно сделать 1 подготовку. 

1.2 если вагон шел из станции А к станции Б, и мы решили сделать ремонт, то вагон должен попасть из пункта А в депо и обратно в пункт А, только после этого можно продолжить маршрут. Работает допущение, что путь в депо происходит мгновенно, но стоит 1000 р за км. 
В качестве усложнения задания вы можете вычислить, сколько занимает маршрут между каждой парой станций и убрать допущение о том, что транспортировка в депо происходит мгновенно. В таком случае, мы не на каждом маршруте можем попасть в депо, а только в тех, где путь из А -> депо -> А -> Б укладывается в расписание. 

1.3 Различные ремонты независимы. Это значит, например, то, что капитальный ремонт не отменяет плановый предупредительный ремонт. 

1.4 Каждый ремонт необходимо повторять раз в 2 года. То есть, если в датасете указано, что ремонт запланирован в 2017-01-01, то его нужно повторить не позднее, чем 2019-01-01.
Мы считаем, что можем оптимизировать только то, что идет после 1-го мая 2018 года. До 1-го мая все ремонты – свершившийся факт, на который мы не можем повлиять

2. Код для вычисления стоимости ремонта для 1 вагона: https://drive.google.com/file/d/1XhgnNK_oEOj1aIMbYKC7fy6VSUE762dm/view?usp=sharing

3. Пример того, как решение выглядит сейчас: https://drive.google.com/file/d/1UH00W1RTztun-vd8agxqcxFSNATPISJh/view?usp=sharing

https://drive.google.com/file/d/1BvalSlBBIRoRLLV7zgOAbJ-b0YneylvY/view?usp=sharing


4. Живое общение с бизнес-оунером процесса.


### Data exploration

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [110]:
depos = pd.read_csv('./data/Депо.csv')
stations = pd.read_csv('./data/Станции.csv')

distances = pd.read_csv('./data/dists.csv')
movements = pd.read_csv('./data/Передвижения вагонов по датам.csv')
repairs = pd.read_csv('./data/Вагоны и плановые ремонты.csv')
repair_prices = pd.read_csv('./data/Плановые ремонты со стоимостями.csv')

Первые два датасета по сути не нужны, они есть в таблице distances.

In [111]:
depos

,id,depo_name
0,1,Депо-Ленинское
1,2,Депо-Комсомольское
2,3,Депо-Молодежное


In [112]:
stations

,id,name
0,1,Сосновка
1,2,Липовка
2,3,Рожок
3,4,Гатка
4,5,Георгополь
5,6,Приморск
6,7,Милта
7,8,Новорепное
8,9,Ясная поляна
9,10,Северный


Дальше идут полезные данные:

In [113]:
repair_prices

,id,repair_name,repair_cost
0,1,Капитальный ремонт,94000000
1,2,Деповской ремонт,19000000
2,3,Плановый предупредительный ремонт,13000000
3,4,Вакуумная очистка и гидроиспытания (ВОиГИ),18000000


In [114]:
distances

,station_id,name,depo_id,depo_name,distance_to_depo
0,1,Сосновка,1,Депо-Ленинское,10077
1,1,Сосновка,2,Депо-Комсомольское,10436
2,1,Сосновка,3,Депо-Молодежное,10522
3,2,Липовка,1,Депо-Ленинское,2084
4,2,Липовка,2,Депо-Комсомольское,3789
5,2,Липовка,3,Депо-Молодежное,7637
6,3,Рожок,1,Депо-Ленинское,9936
7,3,Рожок,2,Депо-Комсомольское,2341
8,3,Рожок,3,Депо-Молодежное,4198
9,4,Гатка,1,Депо-Ленинское,7157


Сразу предобработаем так, чтобы из каждой станции путь был в оптимальное депо:

In [134]:
opt_dist = distances.groupby([pd.Grouper(key='station_id')])['distance_to_depo'].min()
opt_dist = pd.DataFrame(opt_dist)
opt_dist['station_id'] = opt_dist.index
opt_dist.index = np.arange(len(opt_dist))
#df['depo_name'] = distances[df['distance_to_depo'] == distances['distance_to_depo']]['depo_name']
opt_dist

,distance_to_depo,station_id
0,10077,1
1,2084,2
2,2341,3
3,3029,4
4,3662,5
5,2238,6
6,5698,7
7,4816,8
8,1779,9
9,1431,10


In [116]:
movements

,car_num,date,station_id
0,57463085,2015-01-01,2
1,57463085,2015-01-25,6
2,57463085,2015-02-18,9
3,57463085,2015-03-14,5
4,57463085,2015-04-07,8
5,57463085,2015-05-01,2
6,57463085,2015-05-25,6
7,57463085,2015-06-18,9
8,57463085,2015-07-12,3
9,57463085,2015-08-05,6


In [117]:
repairs

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
0,57463085,Газовые,15-78-5,2021-08-08,2018-08-26,2019-08-31,2019-05-19
1,57456220,Газовые,15-78-5,2021-08-02,2019-05-19,2019-09-15,2019-05-19
2,57453656,Газовые,15-78-5,2021-07-19,2019-01-22,2019-07-07,2019-05-18
3,57463507,Газовые,15-78-5,2021-07-19,2019-03-18,2018-06-02,2019-05-18
4,57463309,Газовые,15-78-5,2021-06-20,2018-05-16,2019-01-09,2019-04-24
5,57463341,Газовые,15-78-5,2021-07-11,2018-08-02,2018-06-14,2019-05-20
6,57463275,Газовые,15-78-5,2021-06-20,2018-09-11,2018-10-03,2019-04-22
7,57463317,Газовые,15-78-5,2021-06-14,2018-12-15,2019-06-15,2019-04-22
8,57457038,Газовые,15-78-5,2018-11-25,2018-10-24,2021-11-03,2024-10-24
9,57463523,Газовые,15-78-5,2021-07-01,2019-04-13,2019-02-28,2019-04-22


Посмотрим на пропуски в данных в столбцах ремонтов:

In [118]:
repairs[repairs['std_kap'].isnull()]

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
94,59727719,Нефтебензиновые,11-170-8,NaN,NaN,2018-05-04,NaN
973,59720920,Нефтебензиновые,11-170-8,NaN,NaN,2018-11-01,NaN


In [119]:
repairs[repairs['std_ppr'].isnull()]

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
25,50262674,Нефтебензиновые,19-89-11,2019-04-09,NaN,2018-05-19,NaN
26,50261452,Нефтебензиновые,19-89-11,2019-04-07,NaN,2018-05-26,NaN
27,50262880,Нефтебензиновые,19-89-11,2020-04-30,NaN,2022-04-30,NaN
68,57138265,Нефтебензиновые,16-1347,2019-02-08,NaN,2021-02-08,NaN
72,57063810,Нефтебензиновые,16-1347,2023-02-25,NaN,2020-02-25,NaN
73,57063836,Нефтебензиновые,16-1347,2023-01-19,NaN,2020-01-19,NaN
74,57063851,Нефтебензиновые,16-1347,2017-10-20,NaN,2019-10-20,NaN
75,57063869,Нефтебензиновые,16-1347,2023-04-10,NaN,2019-04-10,NaN
76,57138240,Нефтебензиновые,16-1347,2023-02-23,NaN,2020-02-23,NaN
77,57138257,Нефтебензиновые,16-1347,2017-10-25,NaN,2019-10-25,NaN


In [120]:
repairs[repairs['std_dep'].isnull()]

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi


In [121]:
repairs[repairs['std_vogi'].isnull()]

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
25,50262674,Нефтебензиновые,19-89-11,2019-04-09,NaN,2018-05-19,NaN
26,50261452,Нефтебензиновые,19-89-11,2019-04-07,NaN,2018-05-26,NaN
27,50262880,Нефтебензиновые,19-89-11,2020-04-30,NaN,2022-04-30,NaN
68,57138265,Нефтебензиновые,16-1347,2019-02-08,NaN,2021-02-08,NaN
72,57063810,Нефтебензиновые,16-1347,2023-02-25,NaN,2020-02-25,NaN
73,57063836,Нефтебензиновые,16-1347,2023-01-19,NaN,2020-01-19,NaN
74,57063851,Нефтебензиновые,16-1347,2017-10-20,NaN,2019-10-20,NaN
75,57063869,Нефтебензиновые,16-1347,2023-04-10,NaN,2019-04-10,NaN
76,57138240,Нефтебензиновые,16-1347,2023-02-23,NaN,2020-02-23,NaN
77,57138257,Нефтебензиновые,16-1347,2017-10-25,NaN,2019-10-25,NaN


До конца не понятно, что делать с этими пропусками, ведь мы знаем только по одной дате для каждого ремонта для каждого вагона, то есть не можем "отсчитать" дату для определённого ремонта для данного вагона, если раньше его в данных не было.

Код из выданного ноутбука (стоимость одной ремонтной итерации для одного вагона):

In [122]:
repair_prices = {'std_kap': 94000000, 
                 'std_ppr': 19000000, 
                 'std_dep': 13000000, 
                 'std_vogi': 18000000, 
                 'preparation': 3000000}
price_for_km = 1000

In [123]:
def get_station_from_date(date):
    for index, (car_num, station_date, station_id) in movements.iterrows():
        if station_date < date:
            return station_id
    assert False, 'No stations before this date'

total_repair_price = 0
for repair_type in ['std_kap', 'std_ppr', 'std_dep', 'std_vogi']:
    repair_date = repairs[repair_type].values[0]
    station_id = get_station_from_date(repair_date)
    min_distance_to_depo = distances[distances['station_id'] == 2]['distance_to_depo'].min()
    total_repair_price += 2 * min_distance_to_depo * price_for_km + \
            repair_prices[repair_type] + \
            repair_prices['preparation']

total_repair_price

172672000

Сначала посчитаем, сколько сейчас стоит ремонт для каждого вагона и суммарно (без оптимизаций, следуя дедайнам в исходной таблице):

In [124]:
start_date = '2018-05-01'  # начало оптимизации - 1 мая 2018
end_date = '2018-05-31'  # конец оптимизации - 31 мая 2018

Маска для выделения данных, попадающих во временные рамки:

In [125]:
kap = np.logical_and(repairs['std_kap'] >= start_date, repairs['std_kap'] <= end_date)
dep = np.logical_and(repairs['std_dep'] >= start_date, repairs['std_dep'] <= end_date)
ppr = np.logical_and(repairs['std_ppr'] >= start_date, repairs['std_ppr'] <= end_date)
vogi = np.logical_and(repairs['std_vogi'] >= start_date, repairs['std_vogi'] <= end_date)
mask = np.logical_or(np.logical_or(np.logical_or(kap, dep), ppr), vogi)

In [126]:
actual_repairs = repairs[mask]
actual_repairs

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
4,57463309,Газовые,15-78-5,2021-06-20,2018-05-16,2019-01-09,2019-04-24
16,57456246,Газовые,17-145-5,2018-05-12,2018-07-07,2021-07-28,2024-07-07
21,50815729,Газовые,15-78-5,2018-05-12,2018-11-02,2021-06-04,2024-11-02
24,57459612,Газовые,17-145-5,2018-05-12,2018-10-26,2021-07-31,2024-10-26
25,50262674,Нефтебензиновые,19-89-11,2019-04-09,NaN,2018-05-19,NaN
26,50261452,Нефтебензиновые,19-89-11,2019-04-07,NaN,2018-05-26,NaN
51,57453581,Газовые,17-145-5,2018-05-12,2018-10-22,2021-10-10,2024-10-22
94,59727719,Нефтебензиновые,11-170-8,NaN,NaN,2018-05-04,NaN
173,57738247,Газовые,13-54-6,2020-04-09,2018-04-14,2018-05-16,2018-04-14
185,57751273,Газовые,11-160-9,2020-03-16,2020-04-25,2018-05-08,2026-04-25


In [127]:
REPAIR_TYPES = ['std_kap', 'std_ppr', 'std_dep', 'std_vogi']

In [145]:
def calculate_price_vagon(df, car_num, st_date, e_date):
    car_df = df[df['car_num'] == car_num]
    
    def get_station_from_date(date):
        for index, (car_num, station_date, station_id) in movements.iterrows():
            if station_date < date:
                return station_id

    total_repair_price = 0
    for repair_type in REPAIR_TYPES:
        repair_date = car_df[repair_type].values[0]
        if st_date <= repair_date and repair_date <= e_date:
            station_id = get_station_from_date(repair_date)
            min_distance_to_depo = opt_dist[opt_dist['station_id'] == station_id]['distance_to_depo'].values[0]
            total_repair_price += 2 * min_distance_to_depo * price_for_km + \
                    repair_prices[repair_type] + \
                    repair_prices['preparation']

    return total_repair_price

In [146]:
calculate_price_vagon(actual_repairs, 57463309, start_date, end_date)

26168000

Это число = стоимость ремонтов вагона номер 57463309 в мае 2018, если мы отправили его в самое выгодное (для станции) депо из станции 2 в день 2015-01-01

### Идеи

* Предобработать алгоритмом Флойда пути от станций до других станций (см. другой пункт)
* Написать формулу: оптимизация общей суммы = оптимизация слагаемых
* Есть 24 месяца, то есть ~24 "хороших" варианта ремонтов
* Капитальный ремонт 1 раз в 2 года
* Состояние: (дата, место, какие ремонты делать), метрика (эвристика) хода - расчёт денег при движении вагона из опр. станции в депо в эту дату, учитывающий дедлайновую дату ремонта:  
\__ЭВРИСТИКИ\__ (*А-Star, Метод ветвей и границ, Альфа-бета отсечение*):  
- слишком частые кап ремонты
- много времени до дедлайна ремонта
- уже не идём дальше по ветви, если в вершине большая сумма (больше бейзлайна)
- не укладывается в расписание
- бейзлайн - ремонты в день дедлайна
- невозможность попасть из А в Б на основе просчёта алгоритмом Флойда-Уоршалла
- 

- TASK: сколько мы сэкономили для каждого вагона и в целом
- по неделям
- по вагонам
- общую

### Первая оптимизация - совмещение нескольких типов ремонта в один день

*Идея*: Есть несколько типов ремонта, все обычно проводятся в разные даты. Перегнать вагон в депо стоит денег, подготовить вагон к ремонту тоже стоит денег. Возникает разумная мысль - почему бы не сделать некоторые типы ремонтов в один день, так мы будем экономить на подготовке вагона к ремонту и на логистике - не нужно ездить в депо несколько раз.

### Вторая оптимизация - перегон вагона в другую станцию

*Идея*: В предоставленном коде рассчитывается расстояние от станции до депо, однако станция берётся самая ранняя (`if station_date < date: return station_id`). Но может быть куда выгоднее отправить вагон на ремонт из другой станции, из которой до депо ближе (естественно, если вагон находится в этой станции до даты ремонта). Также будем заботиться о том, чтобы перегон вагона из станции до депо и обратно не мешал вагону отправится из этой станции в следующую.

### Killer-feature

Можно описать всё происходящее как игру...